In [1]:
import pyodbc
from numpy import vectorize
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import BDay
from datetime import datetime
from os import getcwd
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
from UliPlot.XLSX import auto_adjust_xlsx_column_width

SQL

In [2]:
# Get GMLA list and GMLA system date
cklist_confirm_query = """ 
SELECT [projectSizeId]
      ,[PROJECT_SIZE_NAME] as 'Project_Name'
      ,[PROJECT_CATEGORY] as 'Segment'
      ,LEFT([typeName],4) + Right([typeName],1) as 'Phase'
      ,[status]
      /*,[rejectReason]*/
      /*,[kickOffMeetingDate]*/
      ,[baseOverDueDate] as 'actualCloseMeetingDate'
      /*,[submitDate] as 'actualGMLASubmitDate'*/
      /*,[IsByPass]*/
  FROM [PR_WEB2].[dbo].[T_GMLA_CKLIST_CONFIRM]

  LEFT JOIN [PR_WEB2].[dbo].[T_PS_ProdSizeRelateInfo]
  on projectSizeId = PROJECT_SIZE_ID
  /* Join project names and segment */

  LEFT JOIN [PR_WEB2].[dbo].[T_GMLA_BS_VERSION_TYPE]
  ON typeId = [PR_WEB2].[dbo].[T_GMLA_BS_VERSION_TYPE].id
  /* Join GMLA#1/#2/#3 name*/

  WHERE [IsByPass] = '0'
"""

# Get design review date which sync PM system
project_schedule_query = """ 
SELECT [ProjectSizeId] as 'projectSizeId'
      /*
      ,[PROJECT_NAME]
      ,[T_GT_ProjectSchedule].[TASK_ID]
      */
      ,[DESC_OF_TASK]
      /*
      ,[T_GT_TaskBaseInfo].[Note]
      ,[IsPM]
      */
      ,DATEADD(DAY,-21,[Current_Date]) as 'estCloseMeetingDate'
      /* Design review -21 days */
  FROM [PR_WEB2].[dbo].[T_GT_ProjectSchedule]

  LEFT JOIN [PR_WEB2].[dbo].[T_GT_TaskBaseInfo] 
  ON [T_GT_ProjectSchedule].[TASK_ID] = [T_GT_TaskBaseInfo].TASK_ID

  LEFT JOIN [PR_WEB2].[dbo].[T_PS_ProdSizeRelateInfo]
  on ProjectSizeId = [T_PS_ProdSizeRelateInfo].PROJECT_SIZE_ID
  
  WHERE DESC_OF_TASK
  IN ('Mechanical Design Review','Factory Prototype Design Review','Pilot Run Design Review')
"""

In [3]:
connection_string = "DRIVER={SQL Server};SERVER=T1-PE-SUPPORT;DATABASE=PR_WEB2;UID=pieng;PWD=Q2iT5cwHJW3FH"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

cklist_confirm = pd.read_sql(cklist_confirm_query, engine)
project_schedule = pd.read_sql(project_schedule_query, engine, parse_dates='estCloseMeetingDate')

Handle cKlist_confirm

In [4]:
cklist_confirm.loc[cklist_confirm.actualCloseMeetingDate.str.startswith('0001-01', na=False),'actualCloseMeetingDate'] = None

In [5]:
cklist_confirm.actualCloseMeetingDate = pd.to_datetime(cklist_confirm.actualCloseMeetingDate, utc=True).dt.tz_convert('Asia/Taipei')
#cklist_confirm.actualGMLASubmitDate = pd.to_datetime(cklist_confirm.actualGMLASubmitDate,utc=True).dt.tz_convert('Asia/Taipei')
cklist_confirm.head()

,projectSizeId,Project_Name,Segment,Phase,status,actualCloseMeetingDate
0,766,GAD 13,Aviation,GMLA1,0,NaT
1,766,GAD 13,Aviation,GMLA2,0,NaT
2,766,GAD 13,Aviation,GMLA3,0,NaT
3,909,One Battery Bike Project,Fitness,GMLA3,1,2021-02-05 00:00:00+08:00
4,936,Edge 1040,Fitness,GMLA1,1,2021-02-05 00:00:00+08:00


Merge project_schedule

In [6]:
project_schedule.estCloseMeetingDate = project_schedule.estCloseMeetingDate.dt.tz_convert('Asia/Taipei')

In [7]:
project_schedule['Phase'] = project_schedule.DESC_OF_TASK.map(
    {'Mechanical Design Review':'GMLA1','Factory Prototype Design Review':'GMLA2','Pilot Run Design Review':'GMLA3'})
project_schedule.drop_duplicates(subset=['projectSizeId','Phase'], inplace=True)
# Some project have multi-design review date in PMsystem

In [8]:
overall = pd.merge(left=cklist_confirm, right=project_schedule, how='left', on=['projectSizeId','Phase'])
overall.drop('DESC_OF_TASK', axis=1, inplace=True)

In [9]:
def mergeCloseMeeting(actualCloseMeetingDate, estCloseMeetingDate):
    if actualCloseMeetingDate is pd.NaT:
        return estCloseMeetingDate
    else:
        return actualCloseMeetingDate
overall['mergedCloseMeetingDate'] = vectorize(mergeCloseMeeting)(overall.actualCloseMeetingDate, overall.estCloseMeetingDate)
#overall['mergedCloseMeetingDate'] = overall[['actualCloseMeetingDate','estCloseMeetingDate']].apply(lambda overall : mergeCloseMeeting(overall['actualCloseMeetingDate'], overall['estCloseMeetingDate']),axis=1)

In [10]:
overall.iloc[:,5:] = overall.iloc[:,5:].apply(lambda time: pd.to_datetime(time).dt.date)

In [11]:
overall.head()

,projectSizeId,Project_Name,Segment,Phase,status,actualCloseMeetingDate,estCloseMeetingDate,mergedCloseMeetingDate
0,766,GAD 13,Aviation,GMLA1,0,NaT,NaT,NaT
1,766,GAD 13,Aviation,GMLA2,0,NaT,NaT,NaT
2,766,GAD 13,Aviation,GMLA3,0,NaT,NaT,NaT
3,909,One Battery Bike Project,Fitness,GMLA3,1,2021-02-05,2021-07-13,2021-02-05
4,936,Edge 1040,Fitness,GMLA1,1,2021-02-05,2020-10-13,2021-02-05


Load NPI name from PR_web-New Product Status Summary

In [22]:
name = pd.read_excel('New Production Status Summary.xlsx', header=1)
name = name.loc[:,['Project Name','Project Id','SMD PE']]
name.dropna(axis=0, inplace=True)

In [23]:
name['SMD PE'].value_counts()

Shawn.Huang\n                                        13
Anton.Lin\n                                          11
Stephen.Shen\n                                        9
Aaron.Lin\n                                           8
Steve.Chiang\n                                        7
Iris.Wu\n                                             7
Andyjs.Chen\n                                         6
Alex.Pu\n                                             5
Bowman.Lai\n                                          5
Derek.Cheng\nBowman.Lai\nStephen.Shen\n               5
Stephen.Shen\nIris.Wu\nBowman.Lai\n                   4
Dannyhp.Cheng\n                                       4
Derek.Cheng\n                                         4
PeiRu.Tsai\n                                          4
Stephen.Shen\nSteve.Chiang\n                          3
Herry.Lan\nAndyjs.Chen\n                              3
Herry.Lan\n                                           2
Davidsd.Lin\n                                   

In [25]:
name['SMD PE']

0        Davidsd.Lin\nErichw.Wang\n
1                         Alex.Pu\n
2                         Iris.Wu\n
3                    Stephen.Shen\n
4                     Shawn.Huang\n
                   ...             
144                      Emma.Lin\n
145                  Stephen.Shen\n
147                   Shawn.Huang\n
148    Steve.Chiang\nStephen.Shen\n
149        Shawn.Huang\nLionel.Wu\n
Name: SMD PE, Length: 133, dtype: object

In [30]:
name['SMD PE'].apply(lambda name : name.split('\n')[0])

0       Davidsd.Lin
1           Alex.Pu
2           Iris.Wu
3      Stephen.Shen
4       Shawn.Huang
           ...     
144        Emma.Lin
145    Stephen.Shen
147     Shawn.Huang
148    Steve.Chiang
149     Shawn.Huang
Name: SMD PE, Length: 133, dtype: object

Excel writer

In [154]:

#with pd.ExcelWriter(getcwd()+'\data.xlsx', engine='openpyxl') as writer:
    #overall_show.to_excel(writer, sheet_name='overall')
    #auto_adjust_xlsx_column_width(overall_show, writer, sheet_name="overall", margin=5)
    
    #complete_rate.to_excel(writer, sheet_name='complete_rate')
    #auto_adjust_xlsx_column_width(complete_rate, writer, sheet_name="complete_rate", margin=5)
    
    #on_time_rate.to_excel(writer, sheet_name='on_time_rate')
    #auto_adjust_xlsx_column_width(on_time_rate, writer, sheet_name="on_time_rate", margin=5)